# Lab Assignment #6 – Using Maps and Hash Tables

## Exercise 1

**If your first name starts with a letter from A-J inclusively:**

Our `AbstractHashMap` class maintains a load factor l ≤ 0.5. Reimplement
that class to allow the user to specify the maximum load, and adjust the
concrete subclasses accordingly.

Perform experiments on our `ProbeHashMap` classes to measure its
efficiency using random key sets and varying limits on the load factor.
Do you think `ProbeHashMap` is better or `ChainHashMap`? When and how?

**Hint** The load factor can be controlled from within the abstract
class, but there must be means for setting the parameter (either through
the constructor, or a new method).

Write a Java/Python application to test your solution.

In [1]:
from random import randrange
from collections.abc import MutableMapping
import time


class MapBase(MutableMapping):
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
            return self._key < other._key


class HashMapBase(MapBase):
    def __init__(self, cap=11, p=109345121, max_load=0.5):
        self._table = cap * [None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p - 1)
        self._shift = randrange(p)
        self._max_load = max_load

    def _hash_function(self, k):
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) * self._max_load:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def _resize(self, c):
        old = list(self.items())
        self._table = c * [None]
        self._n = 0
        for k, v in old:
            self[k] = v


class ProbeHashMap(HashMapBase):
    _AVAIL = object()

    def _is_available(self, j):
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

    def _find_slot(self, j, k):
        firstAvail = None
        while True:
            if self._is_available(j):
                if firstAvail is None:
                    firstAvail = j
                if self._table[j] is None:
                    return (False, firstAvail)
            elif k == self._table[j]._key:
                return (True, j)
            j = (j + 1) % len(self._table)

    def _bucket_getitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError("Key Error: " + repr(k))
        return self._table[s]._value

    def _bucket_setitem(self, j, k, v):
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k, v)
            self._n += 1
        else:
            self._table[s]._value = v

    def _bucket_delitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError("Key Error: " + repr(k))
        self._table[s] = ProbeHashMap._AVAIL

    def __iter__(self):
        for j in range(len(self._table)):
            if not self._is_available(j):
                yield self._table[j]._key


# Testing the implementation


def test_efficiency(max_load_factors, num_items):
    results = {}
    for max_load in max_load_factors:
        hashmap = ProbeHashMap(max_load=max_load)
        start_time = time.time()

        # Insert random key-value pairs
        for _ in range(num_items):
            key = randrange(10 * num_items)
            value = randrange(10 * num_items)
            hashmap[key] = value

        # Measure insertion time
        insert_time = time.time() - start_time

        # Measure retrieval time
        start_time = time.time()
        for _ in range(num_items):
            key = randrange(10 * num_items)
            try:
                _ = hashmap[key]
            except KeyError:
                pass
        retrieval_time = time.time() - start_time

        results[max_load] = (insert_time, retrieval_time)
    return results


def main():
    max_load_factors = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    num_items = 1000
    results = test_efficiency(max_load_factors, num_items)

    print("Load Factor | Insert Time (s) | Retrieval Time (s)")
    print("-----------------------------------------------")
    for max_load, (insert_time, retrieval_time) in results.items():
        print(f"{max_load:.2f}       | {insert_time:.6f}      | {retrieval_time:.6f}")


if __name__ == "__main__":
    main()

Load Factor | Insert Time (s) | Retrieval Time (s)
-----------------------------------------------
0.10       | 0.044710      | 0.014997
0.20       | 0.048519      | 0.007703
0.30       | 0.050059      | 0.026656
0.40       | 0.035435      | 0.022944
0.50       | 0.021142      | 0.016541
0.60       | 0.015505      | 0.004551
0.70       | 0.017355      | 0.005992
0.80       | 0.013463      | 0.008390
0.90       | 0.015856      | 0.014754


**If your first name starts with a letter from K-Z inclusively:**

Our `AbstractHashMap` class maintains a load factor l ≤ 0.5. Reimplement
that class to allow the user to specify the maximum load, and adjust the
concrete subclasses accordingly.

Perform experiments on our `ChainHashMap` classes to measure its
efficiency using random key sets and varying limits on the load factor.
Do you think `ProbeHashMap` is better or `ChainHashMap`? When and how?

**Hint** The load factor can be controlled from within the abstract
class, but there must be means for setting the parameter (either through
the constructor, or a new method).

Write a Java/Python application to test your solution

In [2]:
import time
from collections.abc import MutableMapping
from random import randint, randrange


class MapBase(MutableMapping):
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
            return self._key < other._key


class HashMapBase(MapBase):
    def __init__(self, cap=11, p=109345121, max_load=0.5):
        self._table = cap * [None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p - 1)
        self._shift = randrange(p)
        self._max_load = max_load

    def _hash_function(self, k):
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) * self._max_load:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def _resize(self, c):
        old = list(self.items())
        self._table = c * [None]
        self._n = 0
        for k, v in old:
            self[k] = v


class UnsortedTableMap(MapBase):
    def __init__(self):
        self._table = []

    def __getitem__(self, k):
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError("Key Error: " + repr(k))

    def __setitem__(self, k, v):
        for item in self._table:
            if k == item._key:
                item._value = v
                return
        self._table.append(self._Item(k, v))

    def __delitem__(self, k):
        for j in range(len(self._table)):
            if k == self._table[j]._key:
                self._table.pop(j)
                return
        raise KeyError("Key Error: " + repr(k))

    def __len__(self):
        return len(self._table)

    def __iter__(self):
        for item in self._table:
            yield item._key


class ChainHashMap(HashMapBase):
    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError("Key Error: " + repr(k))
        return bucket[k]

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:
            self._n += 1

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError("Key Error: " + repr(k))
        del bucket[k]

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:
                for key in bucket:
                    yield key


def test_efficiency(max_load_factors, num_items):
    results = {}
    for max_load in max_load_factors:
        hashmap = ChainHashMap(max_load=max_load)
        start_time = time.time()

        # Insert random key-value pairs
        for _ in range(num_items):
            key = randint(0, 10 * num_items)
            value = randint(0, 10 * num_items)
            hashmap[key] = value

        # Measure insertion time
        insert_time = time.time() - start_time

        # Measure retrieval time
        start_time = time.time()
        for _ in range(num_items):
            key = randint(0, 10 * num_items)
            try:
                _ = hashmap[key]
            except KeyError:
                pass
        retrieval_time = time.time() - start_time

        results[max_load] = (insert_time, retrieval_time)
    return results


def main():
    max_load_factors = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    num_items = 1000
    results = test_efficiency(max_load_factors, num_items)

    print("Load Factor | Insert Time (s) | Retrieval Time (s)")
    print("-----------------------------------------------")
    for max_load, (insert_time, retrieval_time) in results.items():
        print(f"{max_load:.2f}       | {insert_time:.6f}      | {retrieval_time:.6f}")


if __name__ == "__main__":
    main()

Load Factor | Insert Time (s) | Retrieval Time (s)
-----------------------------------------------
0.10       | 0.008712      | 0.001820
0.20       | 0.008864      | 0.004015
0.30       | 0.009454      | 0.002357
0.40       | 0.010426      | 0.004491
0.50       | 0.040042      | 0.003805
0.60       | 0.009975      | 0.001996
0.70       | 0.010087      | 0.001980
0.80       | 0.006440      | 0.002236
0.90       | 0.009488      | 0.003937


- **When to Use ProbeHashMap**:
  - When the expected load factor is low (≤ 0.5).
  - When memory efficiency is a priority.
  - When cache performance is critical.

- **When to Use ChainHashMap**:
  - When the load factor is expected to be high (> 0.5).
  - When uniform performance is needed even with higher load factors.
  - When memory usage is less of a concern compared to consistent performance.

## Exercise 2

**If your first name starts with a letter from A-J inclusively:**

The use of **null** values in a map is problematic, as there is then no
way to differentiate whether a **null** value returned by the call
`get(k)` represents the legitimate value of an entry (k, null), or
designates that key k was not found. The `java.util.Map` interface
includes a boolean method, `containsKey(k)`, that resolves any such
ambiguity.

Implement the `containKey(k)` method for the `SortedTableMap` class.
**Hint:** Use the existing `findIndex(k)` method.

Write a Java/Python application to test your solution.

In [3]:
from collections.abc import MutableMapping


class MapBase(MutableMapping):
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
            return self._key < other._key


class SortedTableMap(MapBase):
    def _find_index(self, k, low, high):
        if high < low:
            return high + 1
        else:
            mid = (low + high) // 2
            if k == self._table[mid]._key:
                return mid
            elif k < self._table[mid]._key:
                return self._find_index(k, low, mid - 1)
            else:
                return self._find_index(k, mid + 1, high)

    def __init__(self):
        self._table = []

    def __len__(self):
        return len(self._table)

    def __getitem__(self, k):
        j = self._find_index(k, 0, len(self._table) - 1)
        if j == len(self._table) or self._table[j]._key != k:
            raise KeyError("Key Error: " + repr(k))
        return self._table[j]._value

    def __setitem__(self, k, v):
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table) and self._table[j]._key == k:
            self._table[j]._value = v
        else:
            self._table.insert(j, self._Item(k, v))

    def __delitem__(self, k):
        j = self._find_index(k, 0, len(self._table) - 1)
        if j == len(self._table) or self._table[j]._key != k:
            raise KeyError("Key Error: " + repr(k))
        self._table.pop(j)

    def __iter__(self):
        for item in self._table:
            yield item._key

    def __reversed__(self):
        for item in reversed(self._table):
            yield item._key

    def find_min(self):
        if len(self._table) > 0:
            return (self._table[0]._key, self._table[0]._value)
        else:
            return None

    def find_max(self):
        if len(self._table) > 0:
            return (self._table[-1]._key, self._table[-1]._value)
        else:
            return None

    def find_le(self, k):
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table) and self._table[j]._key == k:
            return (self._table[j]._key, self._table[j]._value)
        elif j > 0:
            return (
                self._table[j - 1]._key,
                self._table[j - 1]._value,
            )
        else:
            return None

    def find_ge(self, k):
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table):
            return (self._table[j]._key, self._table[j]._value)
        else:
            return None

    def find_lt(self, k):
        j = self._find_index(k, 0, len(self._table) - 1)
        if j > 0:
            return (
                self._table[j - 1]._key,
                self._table[j - 1]._value,
            )
        else:
            return None

    def find_gt(self, k):
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table) and self._table[j]._key == k:
            j += 1
        if j < len(self._table):
            return (self._table[j]._key, self._table[j]._value)
        else:
            return None

    def find_range(self, start, stop):
        if start is None:
            j = 0
        else:
            j = self._find_index(start, 0, len(self._table) - 1)
        while j < len(self._table) and (stop is None or self._table[j]._key < stop):
            yield (self._table[j]._key, self._table[j]._value)
            j += 1

    def containKey(self, k):
        j = self._find_index(k, 0, len(self._table) - 1)
        return j < len(self._table) and self._table[j]._key == k


# Main method to test the solution
def main():
    m = SortedTableMap()
    m["a"] = 1
    m["b"] = 2
    m["c"] = 3

    print("Contain 'a':", m.containKey("a"))  # Expected: True
    print("Contain 'b':", m.containKey("b"))  # Expected: True
    print("Contain 'c':", m.containKey("c"))  # Expected: True
    print("Contain 'd':", m.containKey("d"))  # Expected: False

    del m["b"]
    print("Contain 'b' after deletion:", m.containKey("b"))  # Expected: False


if __name__ == "__main__":
    main()

Contain 'a': True
Contain 'b': True
Contain 'c': True
Contain 'd': False
Contain 'b' after deletion: False


**If your first name starts with a letter from K-Z inclusively:**

Implement the `containKey(k)` method for the `UnSortedTableMap` class.
**Hint:** Use the existing `findIndex(k)` method.

Write a Java/Python application to test your solution.

In [4]:
from collections.abc import MutableMapping


class MapBase(MutableMapping):
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
            return self._key < other._key


class UnsortedTableMap(MapBase):
    def _find_index(self, k, low, high):
        if high < low:
            return high + 1
        else:
            mid = (low + high) // 2
            if k == self._table[mid]._key:
                return mid
            elif k < self._table[mid]._key:
                return self._find_index(k, low, mid - 1)
            else:
                return self._find_index(k, mid + 1, high)

    def __init__(self):
        self._table = []

    def __getitem__(self, k):
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError("Key Error: " + repr(k))

    def __setitem__(self, k, v):
        for item in self._table:
            if k == item._key:
                item._value = v
                return
        self._table.append(self._Item(k, v))

    def __delitem__(self, k):
        for j in range(len(self._table)):
            if k == self._table[j]._key:
                self._table.pop(j)
                return
        raise KeyError("Key Error: " + repr(k))

    def __len__(self):
        return len(self._table)

    def __iter__(self):
        for item in self._table:
            yield item._key

    def containKey(self, k):
        index = self._find_index(k, 0, len(self._table) - 1)
        return index < len(self._table) and self._table[index]._key == k


# Main method to test the solution
def main():
    m = UnsortedTableMap()
    m["a"] = 1
    m["b"] = 2
    m["c"] = 3

    print("Contain 'a':", m.containKey("a"))  # Expected: True
    print("Contain 'b':", m.containKey("b"))  # Expected: True
    print("Contain 'c':", m.containKey("c"))  # Expected: True
    print("Contain 'd':", m.containKey("d"))  # Expected: False

    del m["b"]
    print("Contain 'b' after deletion:", m.containKey("b"))  # Expected: False


if __name__ == "__main__":
    main()

Contain 'a': True
Contain 'b': True
Contain 'c': True
Contain 'd': False
Contain 'b' after deletion: False
